In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch, arrowhead_marker
import seaborn as sns
import warnings
import numpy as np
import ipywidgets as widgets
from highlight_text import  ax_text, fig_text
import matplotlib as mpl

#Read in the data
df = pd.read_csv('Footballdata/Others/Barcelona/Barcelona.csv', encoding='latin-1', low_memory = False)

#Adjust to pitch size
df['x'] = df['x']*1.2
df['y'] = df['y']*.8
df['endX'] = df['endX']*1.2
df['endY'] = df['endY']*.8

#remove
df = df.loc[~(df['type/displayName'] == 'Challenge')]
df = df.loc[~((df['type/displayName'] == 'Aerial') & (df['outcomeType/displayName'] == 'Unsuccessful'))]

#Finding previous event to filter out setpieces
df['prevevent'] = df['type/displayName'].shift(1)

#Remove passes that come after fouls and cards, potential set pieces
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'Foul'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'Card'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'CornerAwarded'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'OffsideGiven'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'SubstitutionOn'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'FormationChange'))]
df = df.loc[~((df['type/displayName'] == 'Pass') & (df['prevevent'] == 'Start'))]

#Filter for passes only
df = df.loc[df['type/displayName'] == 'Pass']

#Passes that reached the penalty area
df['pen']= np.where((df['endY'].between(16.88,63.12)) & ((df['endX']>= 99.6)),1,0)

"""#Passes that reached the penalty area (from outside the box)
df['pen']= np.where(((df['endY'].between(16.88,63.12)) & (df['endX']>= 99.6)) & 
                    ((df['x'] < 99.6) | (df['y'] < 16.88) | (df['y'] > 63.12)),1,0)
"""
df = df.loc[df['pen']== 1]

In [24]:
#Passes originate outside the box
df1 = df.loc[~((df['y'].between(16.88,63.12)) & (df['x'] >= 99.6) & (df['pen']==1))]
sdf1 = df1[df1['outcomeType/displayName']=='Successful']
udf1 = df1[df1['outcomeType/displayName']=='Unsuccessful']

#Passes originate inside the box
df2 = df.loc[(df['y'].between(16.88,63.12)) & (df['x'] >= 99.6) & (df['pen']==1)]
sdf2 = df2[df2['outcomeType/displayName']=='Successful']
udf2 = df2[df2['outcomeType/displayName']=='Unsuccessful']

#List of Pass Attempts
lis = df.groupby(['playername','teamId']).size().reset_index().rename(columns={0:'Attempted Passes'}).sort_values('playername', ascending = True).reset_index()
del lis['index']

#List of Successful Passes
slis1 = sdf1.groupby(['playername','teamId']).size().reset_index().rename(columns={0:'Passes into Box'}).sort_values('playername', ascending = True).reset_index()
del slis1['index']

#List of Successful Passes inside the box
slis2 = sdf2.groupby(['playername','teamId']).size().reset_index().rename(columns={0:'Passes inside Box'}).sort_values('playername', ascending = True).reset_index()
del slis2['index']

#Merge everything, only two at a time
result = pd.merge(lis, slis1, how='outer')
result = pd.merge(result, slis2, how='outer')
result.fillna(0, inplace=True)
result["Attempted Passes"] = result["Attempted Passes"].astype(int)
result["Passes into Box"] = result["Passes into Box"].astype(int)
result["Passes inside Box"] = result["Passes inside Box"].astype(int)

#Display List
result.rename(columns = {'teamId':'Team', 'playername':'Player'},inplace = True)
result = result.sort_values('Passes into Box', ascending= False).reset_index(drop = True)
result.index += 1
result.head(20)

,Player,Team,Attempted Passes,Passes into Box,Passes inside Box
1,Raphinha,Barcelona,75,43,0
2,Jordi Alba,Barcelona,73,33,0
3,Ousmane Dembélé,Barcelona,62,28,0
4,Frenkie de Jong,Barcelona,47,26,0
5,Gavi,Barcelona,44,24,0
6,Pedri,Barcelona,53,22,0
7,Robert Lewandowski,Barcelona,43,21,0
8,Sergio Busquets,Barcelona,24,13,0
9,Alejandro Balde,Barcelona,16,8,0
10,Ferran Torres,Barcelona,20,7,0


In [104]:
#List it in a combo box
options = df["playername"].unique()
options = options.tolist()

options = [x for x in options if str(x) != 'nan']
playerlist = widgets.Combobox(
    placeholder='Enter a name',
    options=options,
    description='Player:',
    ensure_option=True,
    disabled=False
)
playerlist

Combobox(value='', description='Player:', ensure_option=True, options=('Jordi Alba', 'Gonzalo Montiel', 'Raphi…

In [105]:
#Filter for player
player = playerlist.value
df = df[df['playername']== player]

#Passes originate outside the box
df1 = df.loc[~((df['y'].between(18,62)) & (df['x'] >= 102) & (df['pen']==1))]
sdf1 = df1[df1['outcomeType/displayName']=='Successful']
udf1 = df1[df1['outcomeType/displayName']=='Unsuccessful']

numsdf1 = len(sdf1.index)

#Passes originate inside the box
df2 = df.loc[(df['y'].between(18,62)) & (df['x'] >= 102) & (df['pen']==1)]
sdf2 = df2[df2['outcomeType/displayName']=='Successful']
udf2 = df2[df2['outcomeType/displayName']=='Unsuccessful']

numsdf2 = len(sdf2.index)

In [106]:
#Set up pitch
mpl.rcParams['figure.dpi'] = 500
pitch = VerticalPitch(pitch_type='statsbomb', 
                  pitch_color='#000000', line_color='white',half = True,
                  line_zorder=-3, linewidth=8, spot_scale=0.003)

fig,ax = pitch.draw(figsize=(30, 20), constrained_layout=False, tight_layout=True)
fig.set_facecolor('#000000')

#Inverse the y-axis
plt.gca().invert_xaxis()

#Successful Passes (sdf1)
pitch.scatter(sdf1.x,sdf1.y, edgecolors= 'yellow', c='#008140',
              s = 1200, zorder=4, ax=ax, marker = 'o', linewidth=6)

pitch.lines(sdf1.x,sdf1.y,sdf1.endX,sdf1.endY,
            ax=ax, color= '#00c631', lw=13, alpha = 0.9)

pitch.scatter(sdf1.endX,sdf1.endY, edgecolors= 'yellow', c='#008140',marker=arrowhead_marker,
              s = 1500, zorder=4, ax=ax, linewidth=6)


#Successful Passes (sdf2)
pitch.scatter(sdf2.x,sdf2.y, edgecolors= 'yellow', c='#38b4d6',
              s = 1200, zorder=4, ax=ax, marker = 'o', linewidth=6)

pitch.lines(sdf2.x,sdf2.y,sdf2.endX,sdf2.endY,
            ax=ax, color= '#38b4d6', lw=13, alpha = 0.9)

pitch.scatter(sdf2.endX,sdf2.endY, edgecolors= 'yellow', c='#38b4d6',marker=arrowhead_marker,
              s = 1500, zorder=4, ax=ax, linewidth=6)


"""
#Unsuccessful Passes (udf1)
pitch.scatter(udf1.x,udf1.y, edgecolors= 'yellow', c='red',
              s = 1000, zorder=6, ax=ax, marker = 'o', linewidth=6)

pitch.lines(udf1.x,udf1.y,udf1.endX,udf1.endY,
            ax=ax, color= 'red', lw=10, alpha = 0.9)

pitch.scatter(udf1.endX,udf1.endY, edgecolors= 'yellow', c='red',marker=arrowhead_marker,
              s = 1200, zorder=6, ax=ax, linewidth=5)
              
#Unsuccessful Passes (udf2)
pitch.scatter(udf2.x,udf2.y, edgecolors= 'yellow', c='red',
              s = 1000, zorder=6, ax=ax, marker = 'o', linewidth=6)

pitch.lines(udf2.x,udf2.y,udf2.endX,udf2.endY,
            ax=ax, color= 'red', lw=10, alpha = 0.9)

pitch.scatter(udf2.endX,udf2.endY, edgecolors= 'yellow', c='red',marker=arrowhead_marker,
              s = 1200, zorder=6, ax=ax, linewidth=5)
"""
#Title & Annotations
fig.suptitle("\n" + player + "\n\n", fontsize=75, color='yellow',
            fontfamily = "Courier New", fontweight = "bold",y=0.98)

#highlights
highlight_textprops =\
    [{"color": "#00c631","fontweight": 'bold'},
     {"color": "#38b4d6","fontweight": 'bold'}]

fig_text(s = str(numsdf1) + " <Passes Into Box>\n" + str(numsdf2) + " <Passes Inside Box>" ,
        x = 0.5, y = 0.86, highlight_textprops = highlight_textprops ,color='yellow',
        fontfamily = "Courier New",fontweight='bold', ha='center',fontsize=70);

#Space
ax.text(-5, 70, 'space', va='center', ha='left', fontsize=25, color='black',
        fontweight = "bold",fontfamily = "Courier New")
ax.text(89, 70, 'space', va='center', ha='left', fontsize=25, color='black',
        fontweight = "bold",fontfamily = "Courier New")

plt.show()